In [ ]:
!pip install Flask openai

In [ ]:
import os
from flask import Flask, request, jsonify, render_template
import openai

app = Flask(__name__)

# Set your OpenAI API key from an environment variable for security
# On Linux/macOS: export OPENAI_API_KEY='your-api-key-here'
# On Windows: set OPENAI_API_KEY='your-api-key-here'
# Or for a quick test, you can uncomment the line below and replace with your key
# openai.api_key = 'YOUR_API_KEY_HERE' 
openai.api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is set
if not openai.api_key:
    raise ValueError("OPENAI_API_KEY environment variable is not set.")

@app.route('/')
def home():
    """
    Serves the main HTML page for the chatbot.
    """
    return render_template('index.html')

@app.route('/chat', methods=['POST'])
def chat():
    """
    Handles chat requests from the frontend and communicates with the OpenAI API.
    """
    try:
        # Get the user's message from the frontend request
        user_message = request.json.get("message")
        if not user_message:
            return jsonify({"error": "No message provided"}), 400

        print(f"User message: {user_message}")

        # Send the user's message to the OpenAI API
        # We use the ChatCompletion endpoint for conversational AI
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # Or "gpt-4" for more advanced capabilities
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": user_message}
            ]
        )
        
        # Extract the AI's response
        ai_message = response['choices'][0]['message']['content']
        print(f"AI response: {ai_message}")

        # Return the AI's response as a JSON object
        return jsonify({"response": ai_message})

    except openai.error.OpenAIError as e:
        # Handle specific OpenAI API errors
        print(f"OpenAI API Error: {e}")
        return jsonify({"error": "An error occurred with the AI service. Please try again."}), 500
    except Exception as e:
        # Handle other potential errors
        print(f"An unexpected error occurred: {e}")
        return jsonify({"error": "An unexpected error occurred. Please try again."}), 500

if __name__ == '__main__':
    # Run the Flask app
    app.run(debug=True)

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>OpenAI Chatbot</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f9;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
        }
        .chat-container {
            width: 100%;
            max-width: 600px;
            height: 80%;
            background-color: #fff;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            border-radius: 8px;
            display: flex;
            flex-direction: column;
            overflow: hidden;
        }
        .chat-box {
            flex-grow: 1;
            padding: 20px;
            overflow-y: auto;
            border-bottom: 1px solid #ddd;
        }
        .message {
            margin-bottom: 15px;
            display: flex;
            align-items: flex-start;
        }
        .user-message {
            justify-content: flex-end;
        }
        .ai-message {
            justify-content: flex-start;
        }
        .message-bubble {
            padding: 10px 15px;
            border-radius: 20px;
            max-width: 75%;
        }
        .user-message .message-bubble {
            background-color: #007bff;
            color: white;
            border-bottom-right-radius: 5px;
        }
        .ai-message .message-bubble {
            background-color: #e2e6ea;
            color: #333;
            border-bottom-left-radius: 5px;
        }
        .input-container {
            display: flex;
            padding: 10px;
            border-top: 1px solid #ddd;
        }
        #user-input {
            flex-grow: 1;
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 20px;
            outline: none;
            font-size: 16px;
        }
        #send-btn {
            padding: 10px 20px;
            margin-left: 10px;
            background-color: #007bff;
            color: white;
            border: none;
            border-radius: 20px;
            cursor: pointer;
            font-size: 16px;
        }
        #send-btn:hover {
            background-color: #0056b3;
        }
        .loading-dots {
            display: none;
            text-align: center;
            padding: 10px;
        }
        .loading-dots span {
            display: inline-block;
            width: 8px;
            height: 8px;
            background-color: #333;
            border-radius: 50%;
            animation: bounce 1.4s infinite ease-in-out both;
        }
        .loading-dots span:nth-child(1) { animation-delay: -0.32s; }
        .loading-dots span:nth-child(2) { animation-delay: -0.16s; }
        @keyframes bounce {
            0%, 80%, 100% { transform: scale(0); }
            40% { transform: scale(1.0); }
        }
    </style>
</head>
<body>

<div class="chat-container">
    <div class="chat-box" id="chat-box">
        <div class="message ai-message">
            <div class="message-bubble">Hello! How can I help you today?</div>
        </div>
        </div>
    <div class="input-container">
        <input type="text" id="user-input" placeholder="Type your message...">
        <button id="send-btn">Send</button>
    </div>
    <div class="loading-dots" id="loading-dots">
        <span></span><span></span><span></span>
    </div>
</div>

<script>
    document.addEventListener('DOMContentLoaded', () => {
        const chatBox = document.getElementById('chat-box');
        const userInput = document.getElementById('user-input');
        const sendBtn = document.getElementById('send-btn');
        const loadingDots = document.getElementById('loading-dots');

        const appendMessage = (sender, message) => {
            const messageElement = document.createElement('div');
            messageElement.classList.add('message', `${sender}-message`);
            messageElement.innerHTML = `<div class="message-bubble">${message}</div>`;
            chatBox.appendChild(messageElement);
            chatBox.scrollTop = chatBox.scrollHeight; // Auto-scroll to the bottom
        };

        const sendMessage = async () => {
            const message = userInput.value.trim();
            if (message === '') return;

            // Display user message and clear input
            appendMessage('user', message);
            userInput.value = '';

            // Show loading dots while waiting for AI response
            loadingDots.style.display = 'block';
            userInput.disabled = true;
            sendBtn.disabled = true;

            try {
                // Send the message to the Flask backend
                const response = await fetch('/chat', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify({ message: message })
                });

                if (!response.ok) {
                    throw new Error(`HTTP error! status: ${response.status}`);
                }

                const data = await response.json();
                
                // Display the AI's response or an error
                if (data.response) {
                    appendMessage('ai', data.response);
                } else {
                    appendMessage('ai', 'Error: ' + (data.error || 'Something went wrong.'));
                }

            } catch (error) {
                console.error('Fetch error:', error);
                appendMessage('ai', 'An error occurred. Please check the console.');
            } finally {
                // Hide loading dots and re-enable input
                loadingDots.style.display = 'none';
                userInput.disabled = false;
                sendBtn.disabled = false;
                userInput.focus();
            }
        };

        sendBtn.addEventListener('click', sendMessage);
        userInput.addEventListener('keypress', (e) => {
            if (e.key === 'Enter') {
                sendMessage();
            }
        });
    });
</script>

</body>
</html>